In [2]:
import rootutils

rootutils.setup_root("./", indicator=".project-root", pythonpath=True)

import hydra
from hydra.experimental import initialize, compose

initialize(config_path="../configs", job_name="job")
config = compose(config_name="train.yaml", return_hydra_config=True)

/usr/local/lib/python3.11/site-packages/hydra/experimental/initialize.py:43: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(message=message)
/usr/local/lib/python3.11/site-packages/hydra/experimental/initialize.py:45: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  self.delegate = real_initialize(


ValueError: GlobalHydra is already initialized, call GlobalHydra.instance().clear() if you want to re-initialize

In [ ]:
# Load data
from topobenchmarkx.data.load.loaders import HypergraphLoader

data_loader = HypergraphLoader(config)
data = data_loader.load()

Loading hypergraph dataset name: cora
number of nodes:2708, feature dimension: 1433
number of hyperedges: 1072


### Loss

In [3]:
loss = hydra.utils.instantiate(config.model.loss)

### Backbone

In [4]:
hydra.utils.instantiate(config.model.backbone)

AllSetTransformer(
  (layers): ModuleList(
    (0): AllSetTransformerLayer(
      (vertex2edge): AllSetTransformerBlock(
        (multihead_att): MultiHeadAttention()
        (mlp): MLP(
          (0): Linear(in_features=64, out_features=64, bias=False)
          (1): ReLU()
          (2): Dropout(p=0.2, inplace=False)
          (3): Linear(in_features=64, out_features=64, bias=False)
          (4): Dropout(p=0.2, inplace=False)
        )
        (ln0): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
      (edge2vertex): AllSetTransformerBlock(
        (multihead_att): MultiHeadAttention()
        (mlp): MLP(
          (0): Linear(in_features=64, out_features=64, bias=False)
          (1): ReLU()
          (2): Dropout(p=0.2, inplace=False)
          (3): Linear(in_features=64, out_features=64, bias=False)
          (4): Dropout(p=0.2, inplace=False)
        )
        (ln0): LayerNorm((64,), eps=1e-05, ele

### ReadOut

In [4]:
readout = hydra.utils.instantiate(config.model.readout)

In [5]:
readout.parameters()

<generator object Module.parameters at 0x7fe978c5a6c0>

### Evaluator

In [2]:
hydra.utils.instantiate(config.model.evaluation)

### Dataloader

In [3]:
# Load data
from topobenchmarkx.data.load.loaders import HypergraphLoader
from topobenchmarkx.data.dataloader_fullbatch import FullBatchDataModule

data_loader = HypergraphLoader(config)
data = data_loader.load()
dataloader = FullBatchDataModule(data=data)

Loading hypergraph dataset name: cora
number of nodes:2708, feature dimension: 1433
number of hyperedges: 1072


In [5]:
batch = next(iter(dataloader.train_dataloader()))

In [12]:
batch.x[batch.train_mask]

torch.Size([1354, 1433])